In [5]:
from collections import defaultdict
import gzip

iids = []
# in this 1240k iid list, the two african samples are removed
# and one sample not having enough coverage to downsample to 2x is also removed
# leaving a total of 52-3=49 samples
with open('/mnt/archgen/users/yilei/IBDsim/BAM/1240k/iid.list') as f:
    for line in f:
        iid = line.strip()
        iids.append(iid)

mapAllele = defaultdict(set)

for iid in iids:
    with gzip.open(f'../realMosaicSim_wgs/genotypes/{iid}.chr3.1240k.vcf.gz', 'rt') as f:
        for line in f:
            if line.startswith('#'):
                continue
            _, bp, _, ref, alt, _, _, _, _, record = line.strip().split()
            bp = int(bp)
            GT = record.split(':')[0]
            gt = int(GT[0]) + int(GT[2])
            mapAllele[bp].add(gt)

from collections import Counter
c = Counter([len(v) for _, v in mapAllele.items()])
print(c)


Counter({3: 50588, 2: 14200, 1: 12864})


In [9]:
# impute accu map
path2sites="/mnt/archgen/users/hringbauer/data/GRG/vcf/pos/chr3.sites"
iids = []
with open('/mnt/archgen/users/yilei/IBDsim/BAM/1240k/iid.list') as f:
    for line in f:
        iid = line.strip()
        iids.append(iid)

import pickle
import os

for cov in ['cov3', 'cov2', 'cov1', 'cov3over4', 'cov1over2', 'cov1over4', 'cov1over10']:
    count = 0
    accuMap = {}
    with open(path2sites) as f:
        for line in f:
            _, bp = line.strip().split()
            bp = int(bp)
            accuMap[bp] ={0:[], 1:[], 2:[]}

    nsample = 0
    for iid in iids:
        if not os.path.exists(f'./downsample/{cov}/{iid}/ch3/GLIMPSE_ligated/{iid}.chr3.1240k.vcf'):
            continue
        # first, determine groundtruth genotype at each site
        nsample += 1
        genotypes = {}
        with gzip.open(f'../realMosaicSim_wgs/genotypes/{iid}.chr3.1240k.vcf.gz', 'rt') as f:
            for line in f:
                if line.startswith('#'):
                    continue
                _, bp, _, ref, alt, _, _, _, _, record = line.strip().split()
                bp = int(bp)
                GT, GP = record.split(':')
                gt = int(GT[0]) + int(GT[2])
                genotypes[bp] = gt
        with open(f'./downsample/{cov}/{iid}/ch3/GLIMPSE_ligated/{iid}.chr3.1240k.vcf') as f:
            for line in f:
                if line.startswith('#'):
                    continue
                _, bp, _, ref, alt, _, _, _, _, record = line.strip().split()
                bp = int(bp)
                GT, _, GP, _ = record.split(':')
                gt = int(GT[0]) + int(GT[2])
                if gt != genotypes[bp]:
                    count += 1
                    #     print(f'inconsistent imputed genotype at {bp}')
                gp0, gp1, gp2 = GP.split(',')
                gp0, gp1, gp2 = float(gp0), float(gp1), float(gp2)
                accuMap[bp][genotypes[bp]].append((gp0, gp1, gp2))
            
    pickle.dump(accuMap, open(f'imputeAccuMap.{cov}', 'wb'))
    print(f'{count/nsample} GT switches per sample for {cov}')   

1077.925 GT switches per sample for cov3
1492.2857142857142 GT switches per sample for cov2
2860.4489795918366 GT switches per sample for cov1
3793.877551020408 GT switches per sample for cov3over4
5427.183673469388 GT switches per sample for cov1over2
9273.612244897959 GT switches per sample for cov1over4
15078.938775510203 GT switches per sample for cov1over10


In [ ]:
import pickle

accuMap = pickle.load(open('imputeAccuMap.cov1over2', 'rb'))

for bp, map_ = accuMap.items():
    

In [4]:
import pandas as pd
dat = pd.read_csv('/mnt/archgen/users/yilei/IBDsim/realMosaicSim_1240k/calledIBD/cov1/ch3_0cm/ch3.tsv', sep='\t')
dat = dat[dat['lengthM']>0.04]
print(len(dat))
dat = dat[(dat['End'] - dat['Start'])/(dat['lengthM']*100) >= 220]
print(len(dat))

409
249
